# Fase 2: Preprocesamiento y Selección de Variables

In [242]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
import joblib
import json
import hashlib
from datetime import datetime


In [243]:
# carga de datos
df_raw = pd.read_csv('../data/df_raw.csv', index_col=0, parse_dates=True)
target_col = 'Consolidado EE_Frio (Kw)'

print(f"Dataset crudo: {df_raw.shape[0]} días × {df_raw.shape[1]} columnas")
print(f"Rango: {df_raw.index.min()} -> {df_raw.index.max()}")

Dataset crudo: 1185 días × 295 columnas
Rango: 2020-07-01 00:00:00 -> 2023-10-25 00:00:00


## 2.1. Limpieza de Datos

In [244]:
# tratamiento de errores 

# reemplazar errores comunes de Excel
errors = ['#VALUE!', '#DIV/0!', '#REF!', '#N/A', '#NAME?', '#NUM!', '#NULL!']
df = df_raw.replace(errors, np.nan, regex=True)

# Convertir a numérico 
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [245]:
# eliminar outliers extremos
sane_threshold = 40000  # kW
outliers_before = len(df)

df = df[df[target_col] < sane_threshold].copy()

print(f"Outliers >{sane_threshold:,} kW eliminados: {outliers_before - len(df)}")
print(f"Dataset tras limpieza: {df.shape}")

Outliers >40,000 kW eliminados: 24
Dataset tras limpieza: (1161, 295)


In [246]:
# eliminar columnas con >80% missing o 'Unnamed'
missing_pct = df.isna().mean()
high_missing = missing_pct[missing_pct > 0.8].index
unnamed_cols = df.columns[df.columns.str.contains('Unnamed', case=False)]

cols_to_drop = high_missing.union(unnamed_cols)
df = df.drop(columns=cols_to_drop)

print(f"Columnas con >80% missing: {len(high_missing)}")
print(f"Columnas Unnamed: {len(unnamed_cols)}")
print(f"Total eliminadas: {len(cols_to_drop)}")
print(f"Dataset tras limpieza: {df.shape}")

Columnas con >80% missing: 0
Columnas Unnamed: 1
Total eliminadas: 1
Dataset tras limpieza: (1161, 294)


In [247]:
# imputación de valores faltantes
missing_before = df.isna().sum().sum()
print(f"Valores faltantes antes de imputación: {missing_before}")
df = df.sort_index()

# 1. Forward fill (serie temporal)
df = df.ffill()

# 2. Backward fill (para huecos al inicio)
df = df.bfill()

# 3. Mediana como fallback
df = df.fillna(df.median(numeric_only=True))

missing_after = df.isna().sum().sum()
print(f"Valores faltantes tras imputación: {missing_after}")

Valores faltantes antes de imputación: 10021
Valores faltantes tras imputación: 0


## 2.2. Feature Engineering

In [248]:
# creación de variables temporales

# extraer componentes de fecha
date_features = pd.DataFrame({
    'day_of_week': df.index.dayofweek,    # 0 = lunes
    'month': df.index.month,
    'is_weekend': df.index.dayofweek.isin([5, 6]).astype(int)
}, index=df.index)

df = pd.concat([df, date_features], axis=1)

print("Variables temporales creadas: dia de la semana, mes, es fin de semana")
print(f"Shape final: {df.shape}")

Variables temporales creadas: dia de la semana, mes, es fin de semana
Shape final: (1161, 297)


In [249]:
# crear lags y rolling statistics
df['lag_1'] = df[target_col].shift(1)
df['lag_7'] = df[target_col].shift(7)
df['rolling_7'] = df[target_col].rolling(window=7).mean()
df['rolling_30'] = df[target_col].rolling(window=30).mean()


In [250]:
# crear variable de producción total (Hl) y eficiencia por Hl
hl_cols = [col for col in df.columns if 'Hl ' in col and 'Most' not in col]
df['hl_total'] = df[hl_cols].sum(axis=1)

df['ee_frio_por_hl'] = df[target_col] / (df['hl_total'] + 1e-6)  # evitar división por 0

print(f"Producción total (Hl): {len(hl_cols)} columnas sumadas")

Producción total (Hl): 13 columnas sumadas


In [251]:
# crear variable de eficiencia energética (real vs meta)

# búsqueda flexible de columnas "Meta" relacionadas con Frío o KPI
meta_candidates = [
    col for col in df.columns 
    if any(keyword in col.lower() for keyword in ['meta', 'kpi', 'objetivo'])
    and any(area in col.lower() for area in ['frio', 'ee', 'planta'])
]

if meta_candidates:
    # Tomamos promedio de todas las metas relevantes
    df['meta_frio_promedio'] = df[meta_candidates].mean(axis=1, skipna=True)
    df['eficiencia_frio'] = df['meta_frio_promedio'] / (df[target_col] + 1e-6)
    print(f"Meta detectada: {len(meta_candidates)} columnas")
    print(f"  → {meta_candidates[:3]}...")
    print("  → 'eficiencia_frio' = meta_promedio / consumo_real")
else:
    # Si no hay meta, usar ratio por Hl 
    hl_cols = [c for c in df.columns if 'Hl ' in c]
    if hl_cols:
        df['hl_total'] = df[hl_cols].sum(axis=1)
        df['eficiencia_frio'] = df[target_col] / (df['hl_total'] + 1e-6)
        print("No hay 'Meta', entonces usando kW/Hl como proxy de eficiencia")
    else:
        df['eficiencia_frio'] = 1.0  # fallback
        print("No hay datos para eficiencia y por lo tanto, valor 1.0")

Meta detectada: 22 columnas
  → ['Consolidado KPI_EE Planta / Hl', 'Consolidado KPI_EE Elaboracion / Hl', 'Consolidado KPI_EE Bodega / Hl']...
  → 'eficiencia_frio' = meta_promedio / consumo_real


In [252]:
# creación de interacciones
interacciones = []

# 1. Resto Serv × Mycom 7
col_resto = 'Consolidado EE_Resto Serv (Kw)'
col_mycom7 = next((c for c in df.columns if 'Mycom 7' in c), None)

if col_resto in df.columns and col_mycom7:
    df['inter_resto_x_mycom7'] = df[col_resto] * df[col_mycom7]
    interacciones.append('resto × mycom7')
else:
    print(f"Advertencia: No se encontró '{col_mycom7}' para interacción")

# 2. Agua Condensada × Frio
col_agua = next((c for c in df.columns if 'agua' in c.lower() and 'cond' in c.lower()), None)

if col_agua and target_col in df.columns:
    df['inter_agua_cond_x_frio'] = df[col_agua] * df[target_col]
    interacciones.append('agua_cond × frio')
else:
    print(f"Advertencia: No se encontró columna de agua condensada")

# 3. Fallback: interacción con driver más correlacionado
if len(interacciones) == 0:
    top_corr = 'Consolidado EE_Sala Maq (Kw)'
    if top_corr in df.columns:
        df['inter_fallback'] = df[top_corr] * df[target_col]
        interacciones.append('sala_maq × frio')
        print("Usando fallback: Sala Maq × Frio")

print(f"Interacciones creadas: {len(interacciones)} -> {interacciones}")

Interacciones creadas: 2 -> ['resto × mycom7', 'agua_cond × frio']


## 2.3 Selección de Variables

In [ ]:
# creación de target y split temporal

target_col = 'Consolidado EE_Frio (Kw)'

# 1. Crear target = Frio(t+1)
df['target'] = df[target_col].shift(-1)

# 2. Eliminar filas con NaN (por shift, rolling, etc.)
print(f"Filas antes de dropna(): {len(df)}")
df = df.dropna()
print(f"Filas después de dropna(): {len(df)}")

# 3. Verificación
assert df.isna().sum().sum() == 0, "Toda hay NaN"
print("0 NaN → listo para modelado")

# 4. Split temporal
train = df[df.index.year <= 2022].copy()
test = df[df.index.year == 2023].copy()

print(f"Train: {len(train)} días")
print(f"Test:  {len(test)} días")

# 5. Separar X/y y excluir Frio(t)
X_train = train.drop(columns=['target', target_col])  # ¡EXCLUIR FRIO(t)!
y_train = train['target']

X_test = test.drop(columns=['target', target_col])
y_test = test['target']

print(f"Features shape (sin Frio(t)): {X_train.shape}")

Filas antes de dropna(): 1161
Filas después de dropna(): 1131
0 NaN → listo para modelado
Train: 840 días
Test:  291 días
Features shape (sin Frio(t)): (840, 306)


In [254]:
# 2.3.2 - Random Forest para importancia de variables (sin Frio(t))
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

importances = pd.Series(rf.feature_importances_, index=X_train.columns)
top_15_rf = importances.nlargest(15)

print("TOP 15 (Random Forest):")
print(top_15_rf.round(4))

TOP 15 (Random Forest):
rolling_7                                        0.5518
Consolidado EE_Sala Maq (Kw)                     0.1269
Consolidado EE_Servicios (Kw)                    0.0374
Servicios_total                                  0.0276
inter_agua_cond_x_frio                           0.0116
rolling_30                                       0.0069
Consolidado EE_Planta (Kw)                       0.0055
Totalizadores Energia_KW Gral Planta             0.0042
Totalizadores Energia_KW Laboratorio             0.0041
Totalizadores Energia_KW Linea 3                 0.0038
Totalizadores Energia_KW Cond 5. 6 y 9           0.0035
eficiencia_frio                                  0.0033
Consolidado EE_Restos Planta (Kw)                0.0030
Totalizadores Energia_KW Obrador Contratistas    0.0029
Consolidado EE_KW Gral Planta                    0.0028
dtype: float64


In [255]:
# 2.3.3 - RFE (sin Frio(t))

model = LinearRegression()
rfe = RFE(estimator=model, n_features_to_select=15)
rfe.fit(X_train, y_train)

selected_rfe = X_train.columns[rfe.support_].tolist()

print(f"RFE seleccionó {len(selected_rfe)} variables:")
for i, col in enumerate(selected_rfe, 1):
    print(f"  {i:2d}. {col}")

RFE seleccionó 15 variables:
   1. Consolidado KPI_EE Planta / Hl
   2. Consolidado KPI_EE Elaboracion / Hl
   3. Consolidado KPI_EE Bodega / Hl
   4. Consolidado KPI_EE Envasado / Hl
   5. Consolidado KPI_EE Servicios / Hl
   6. Consolidado KPI_EE Frio / Hl
   7. Consolidado KPI_EE Aire / Hl
   8. Consolidado KPI_EE Caldera / Hl
   9. Consolidado KPI_EE Agua / Hl
  10. Consolidado KPI_EE Resto Serv / Hl
  11. Consolidado KPI_EE Resto Planta / Hl
  12. Consolidado KPI_Agua Planta / Hl
  13. Consolidado KPI_Aire Planta / Hl
  14. Consolidado KPI_CO 2 linea 3 / Hl
  15. Consolidado KPI_CO 2 Linea 4 / Hl


In [256]:
# 2.3.4 - Conjunto final (unión RF + RFE)

final_features = list(set(top_15_rf.index) | set(selected_rfe))

# eliminar target_col si está presente 
if target_col in final_features:
    final_features.remove(target_col)

print(f"Variables finales: {len(final_features)}")
for i, col in enumerate(sorted(final_features), 1):
    print(f"  {i:2d}. {col}")


# Dataset final (solo features + target(t+1))
df_final = df[final_features + ['target']].copy()
print(f"Dataset final shape: {df_final.shape}")

Variables finales: 30
   1. Consolidado EE_KW Gral Planta
   2. Consolidado EE_Planta (Kw)
   3. Consolidado EE_Restos Planta (Kw)
   4. Consolidado EE_Sala Maq (Kw)
   5. Consolidado EE_Servicios (Kw)
   6. Consolidado KPI_Agua Planta / Hl
   7. Consolidado KPI_Aire Planta / Hl
   8. Consolidado KPI_CO 2 Linea 4 / Hl
   9. Consolidado KPI_CO 2 linea 3 / Hl
  10. Consolidado KPI_EE Agua / Hl
  11. Consolidado KPI_EE Aire / Hl
  12. Consolidado KPI_EE Bodega / Hl
  13. Consolidado KPI_EE Caldera / Hl
  14. Consolidado KPI_EE Elaboracion / Hl
  15. Consolidado KPI_EE Envasado / Hl
  16. Consolidado KPI_EE Frio / Hl
  17. Consolidado KPI_EE Planta / Hl
  18. Consolidado KPI_EE Resto Planta / Hl
  19. Consolidado KPI_EE Resto Serv / Hl
  20. Consolidado KPI_EE Servicios / Hl
  21. Servicios_total
  22. Totalizadores Energia_KW Cond 5. 6 y 9
  23. Totalizadores Energia_KW Gral Planta
  24. Totalizadores Energia_KW Laboratorio
  25. Totalizadores Energia_KW Linea 3
  26. Totalizadores Energi

## 2.4. Preparación y Versionado de Datos Procesados

In [257]:
# 2.4.1 - Escalado (RobustScaler)
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train[final_features])
X_test_scaled = scaler.transform(X_test[final_features])

print(f"X_train_scaled: {X_train_scaled.shape}")
print(f"X_test_scaled:  {X_test_scaled.shape}")

# Guardar scaler
os.makedirs('../models', exist_ok=True)
joblib.dump(scaler, '../models/scaler.pkl')


X_train_scaled: (840, 30)
X_test_scaled:  (291, 30)


['../models/scaler.pkl']

In [258]:
# 2.5.1 - Guardar datasets procesados

os.makedirs('../data/processed', exist_ok=True)

# 1. dataset_final.csv (sin escalar)
dataset_final = df_final.copy()  # ya tiene final_features + 'target'
dataset_final.to_csv('../data/processed/dataset_final.csv')
print("dataset_final.csv guardado (sin escalar)")

# 2. X_train, X_test, y_train, y_test (escalados)
pd.DataFrame(X_train_scaled, columns=final_features).to_csv('../data/processed/X_train.csv', index=False)
pd.DataFrame(X_test_scaled, columns=final_features).to_csv('../data/processed/X_test.csv', index=False)
pd.Series(y_train).to_csv('../data/processed/y_train.csv', header=['target'], index=False)
pd.Series(y_test).to_csv('../data/processed/y_test.csv', header=['target'], index=False)

print("X_train, X_test, y_train, y_test guardados (escalados)")

dataset_final.csv guardado (sin escalar)
X_train, X_test, y_train, y_test guardados (escalados)


In [259]:
# 3. linaje y checksum

# Linaje
lineage = {
    "fecha": datetime.now().isoformat(),
    "script": "preprocesamiento.ipynb",
    "rama": "feature/preprocessing",
    "outliers": ">40,000 kW eliminados",
    "imputacion": "ffill + bfill + mediana",
    "feature_engineering": ["lag_7", "rolling_7", "rolling_30", "eficiencia_frio", "interacciones"],
    "seleccion": "RF + RFE → 29 variables",
    "split": "Train ≤2022, Test 2023",
    "shapes": {
        "X_train": X_train_scaled.shape,
        "X_test": X_test_scaled.shape
    },
    "archivos_guardados": [
    "dataset_final.csv",
    "X_train.csv", "X_test.csv",
    "y_train.csv", "y_test.csv",
    "scaler.pkl"
]
}

with open('../data/processed/data_lineage.json', 'w') as f:
    json.dump(lineage, f, indent=2)

# Checksum
def md5(obj):
    return hashlib.md5(str(obj).encode()).hexdigest()

checksums = {
    "X_train": md5(X_train_scaled.tolist()),
    "X_test": md5(X_test_scaled.tolist()),
    "y_train": md5(y_train.tolist()),
    "y_test": md5(y_test.tolist())
}

with open('../data/checksums.json', 'w') as f:
    json.dump(checksums, f, indent=2)

print("Linaje y checksum guardados")

Linaje y checksum guardados
